<a href="https://colab.research.google.com/github/leekh8/Colaboratory/blob/main/ex10_MLP%2CCNN_%EA%B0%9C%2C%EA%B3%A0%EC%96%91%EC%9D%B4_%EC%9D%B4%EC%A7%84%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool/'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Colaboratory/AISchool


# 목표
- 개, 고양이 데이터 이미지를 배열로 변환 (이미지 데이터 전처리)
  - 이미지 자체를 저장소에 업로드하는 것 보다 배열로 변환했을 때 용량 소모 적음
- MLP 이미지 데이터 이진 분류
- CNN 이미지 데이터 이진 분류
- 성능 향상을 위한 노력
  - 이미지 증식
  - 전이학습

In [2]:
# 환경 세팅
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 이미지 다루는 라이브러리
from PIL import Image

In [3]:
# 개고양이 배열 데이터 불러오기
data = np.load('./Data/np_cat_vs_dog.npz')
data

In [4]:
len(data)

4

In [5]:
# 각각의 변수에 데이터 분리해서 담아주기
X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']

# 크기 확인
print("훈련 셋: ", X_train.shape, y_train.shape)
print("테스트 셋: ", X_test.shape, y_test.shape)

훈련 셋:  (2000, 224, 224, 3) (2000,)
테스트 셋:  (1000, 224, 224, 3) (1000,)


# MLP 모델 생성

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
# 1. 모델 설계
# 뼈대 구축
mlp_model = Sequential()

# 입력 (3차원 -> 1차)
mlp_model.add(Flatten(input_shape = (224, 224, 3)))

# 중간층
mlp_model.add(Dense(units = 256, activation = 'relu'))
mlp_model.add(Dense(units = 128, activation = 'relu'))
mlp_model.add(Dense(units = 64, activation = 'relu'))

# 출력층 (분류)
mlp_model.add(Dense(units = 1, activation = 'sigmoid'))
mlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 256)               38535424  
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 38576641 (147.16 MB)
Trainable params: 38576641 (147.16 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 번역 compile
# pc가 이해하도록 번역
# loss, optimizer, metrics 설정

# 이진 분류
mlp_model.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ["accuracy"])

In [10]:
# 조기 학습 중단 설정
# 검증 정확도, 인내심 5번
f_ea = EarlyStopping(monitor = 'val_accuracy', patience = 5)

In [11]:
# 학습, 검증 데이터 분리 (7:3), batch_size = 64, 반복 횟수 = 50, 조기 학습 중단 연결
mlp_his = mlp_model.fit(X_train, y_train, validation_split = 0.3, epochs = 50, batch_size = 64, callbacks = [f_ea])

Epoch 1/50
22/22 [==============================] - 7s 40ms/step - loss: 2252.6592 - accuracy: 0.5971 - val_loss: 85.2277 - val_accuracy: 0.6183
Epoch 2/50
22/22 [==============================] - 1s 23ms/step - loss: 871.0434 - accuracy: 0.5936 - val_loss: 3033.7542 - val_accuracy: 0.0000e+00
Epoch 3/50
22/22 [==============================] - 0s 23ms/step - loss: 587.8466 - accuracy: 0.6271 - val_loss: 297.0696 - val_accuracy: 0.2917
Epoch 4/50
22/22 [==============================] - 0s 20ms/step - loss: 266.6433 - accuracy: 0.6350 - val_loss: 18.3940 - val_accuracy: 0.9150
Epoch 5/50
22/22 [==============================] - 0s 23ms/step - loss: 269.5626 - accuracy: 0.6129 - val_loss: 2199.4504 - val_accuracy: 0.0000e+00
Epoch 6/50
22/22 [==============================] - 1s 23ms/step - loss: 403.7193 - accuracy: 0.6357 - val_loss: 1814.8420 - val_accuracy: 0.0000e+00
Epoch 7/50
22/22 [==============================] - 1s 23ms/step - loss: 353.9066 - accuracy: 0.6250 - val_loss: 306

In [12]:
# test 데이터 활용해 평가하기
# 모델.score(문제, 답) -> 머신 러닝 모델 평가 함수 (sklearn)
# 모델.evaluate(문제, 답) -> 딥러닝 모델 평가 함수 (tensorflow.keras)
mlp_model.evaluate(X_test, y_test)

32/32 [==============================] - 0s 5ms/step - loss: 121.2461 - accuracy: 0.5650


[121.24605560302734, 0.5649999976158142]

# CNN 모델 생성
- 1. 모델 설계 (CNN 층)
- 2. 모델 학습 및 평가 방법 설정
- 3. 모델 학습 및 시각화
- 4. 모델 평가 및 예측

In [14]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout

In [15]:
# cnn 신경망 구조 설계
# 뼈대 생성
cnn_model = Sequential()

# 대상에 집중 -> 중요한 특성에 집중 -> 특성 추출
# 특성 추출부(층) - conv(합성곱)
cnn_model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = (224, 224, 3), activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

cnn_model.add(Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
cnn_model.add(MaxPooling2D(pool_size = 2))

# 고양이, 개 분류부(층) - mlp(입력(1차원으로 펴주는 기능), 중간, 출력층(이진 분류))
cnn_model.add(Flatten())
cnn_model.add(Dense(512, activation = 'relu'))
cnn_model.add(Dense(1, activation = 'sigmoid'))

# 모델 정보 확인
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 128)      

In [17]:
# compile
cnn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics= ['accuracy'])

# fit
cnn_his = cnn_model.fit(X_train, y_train, validation_split= 0.3, batch_size = 64, epochs = 50, callbacks= [f_ea])

Epoch 1/50
22/22 [==============================] - 13s 202ms/step - loss: 249.6688 - accuracy: 0.5507 - val_loss: 0.7053 - val_accuracy: 0.0983
Epoch 2/50
22/22 [==============================] - 2s 108ms/step - loss: 0.6388 - accuracy: 0.7100 - val_loss: 1.2939 - val_accuracy: 0.0033
Epoch 3/50
22/22 [==============================] - 2s 107ms/step - loss: 0.5228 - accuracy: 0.7464 - val_loss: 1.1300 - val_accuracy: 0.3767
Epoch 4/50
22/22 [==============================] - 2s 106ms/step - loss: 0.4290 - accuracy: 0.8157 - val_loss: 1.6260 - val_accuracy: 0.2650
Epoch 5/50
22/22 [==============================] - 2s 102ms/step - loss: 0.3003 - accuracy: 0.8721 - val_loss: 2.0889 - val_accuracy: 0.3600
Epoch 6/50
22/22 [==============================] - 2s 107ms/step - loss: 0.2034 - accuracy: 0.9164 - val_loss: 2.5003 - val_accuracy: 0.2833
Epoch 7/50
22/22 [==============================] - 2s 106ms/step - loss: 0.1453 - accuracy: 0.9536 - val_loss: 7.3087 - val_accuracy: 0.1717
Epo

In [18]:
# 과대적합 보여짐
# 왜 이렇게 학습했을까?
# cnn 틀성에 집중 -> val 예측 성능은 떨어지는 상태
# 데이터는 복잡한 데이터일 수 있읍
# 반대로 모델은 너무 단순할 수 있음

# 평가
cnn_model.evaluate(X_test, y_test)[1]

32/32 [==============================] - 1s 18ms/step - loss: 2.2117 - accuracy: 0.5800


0.5799999833106995